In [1]:
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
from tkinter import messagebox

    
import io
import shutil, os
from PIL import Image, ExifTags
from geopy.geocoders import Nominatim
from PIL.ExifTags import GPSTAGS
from PIL.ExifTags import TAGS
import pandas as pd
import datetime
from datetime import datetime
import folium
from folium.plugins import MarkerCluster
from PyQt5 import QtWidgets, QtWebEngineWidgets

gui = Tk()
gui.geometry("600x300")
gui.title("Digital Photo Organizer")

def get_exif(filename):
    image = Image.open(filename)
    image.verify()
    return image._getexif()


def get_labeled_exif(exif):
    labeled = {}
    for (key, val) in exif.items():
        labeled[TAGS.get(key)] = val

    return labeled

def get_geotagging(exif):
    if not exif:
        raise ValueError("No EXIF metadata found")

    geotagging = {}
    for (idx, tag) in TAGS.items():
        if tag == 'GPSInfo':
            if idx in exif:
                for (key, val) in GPSTAGS.items():
                    if key in exif[idx]:
                        geotagging[val] = exif[idx][key]
    return geotagging

def get_decimal_from_dms(dms, ref):

    degrees = dms[0][0] / dms[0][1]
    minutes = dms[1][0] / dms[1][1] / 60.0
    seconds = dms[2][0] / dms[2][1] / 3600.0

    if ref in ['S', 'W']:
        degrees = -degrees
        minutes = -minutes
        seconds = -seconds

    return round(degrees + minutes + seconds, 5)

def get_coordinates(geotags):
    if ('GPSLatitude') in geotags:
        lat = get_decimal_from_dms(geotags['GPSLatitude'], geotags['GPSLatitudeRef'])

        lon = get_decimal_from_dms(geotags['GPSLongitude'], geotags['GPSLongitudeRef'])
    else:
        lat = 0;
        lon = 0;        
    return (lat,lon)

def create_df():
    global img_data_df

    folder_path=  folderPath.get() + '/'
    files = os.listdir(folder_path)

    # Create dataframe from exif image information
    header = ['name', 'date','location', 'lat', 'lon', 'geo', 'zipcode', 'exif']
    img_data = []

    for file in files:
        row_dict = {}
        row_dict['name'] = file
        filename = folder_path+file
        img_exif = get_exif(filename)
        if img_exif:
            exif = { ExifTags.TAGS[k]: v for k, v in img_exif.items() if k in ExifTags.TAGS and type(v) is not bytes }
            row_dict['exif'] = exif
            date_obj = datetime.strptime(exif['DateTimeOriginal'], '%Y:%m:%d %H:%M:%S')
            row_dict['date'] = date_obj
            geotags = get_geotagging(img_exif)
            if (len(geotags)>0):
                locator = Nominatim(user_agent='myGeocoder')
                lat, lon = get_coordinates(geotags)
                row_dict['lat'] = lat
                row_dict['lon'] = lon            
                coordinates = str(lat) + ', ' + str(lon)
                location = locator.reverse(coordinates)
                row_dict['geo'] = get_coordinates(geotags)

                if ('postcode' in location.raw['address']):
                    row_dict['zipcode'] = location.raw['address']['postcode']
                
                if ('locality' in location.raw['address']):
                    row_dict['location'] = location.raw['address']['locality']            
                if ('city' in location.raw['address']):
                    row_dict['location'] = location.raw['address']['city']
                if ('town' in location.raw['address']):
                    row_dict['location'] = location.raw['address']['town']            
                if ('hamlet' in location.raw['address']):
                    row_dict['location'] = location.raw['address']['hamlet']
                if ('retail' in location.raw['address']):
                    row_dict['location'] = location.raw['address']['retail']            
                if ('suburb' in location.raw['address']):
                    row_dict['location'] = location.raw['address']['suburb']            
             
            else:
                row_dict['geo'] = 'N/A'
            

        else:
            row_dict['date'] = 'N/A'

        img_data.append(row_dict)
        
    img_data_df = pd.DataFrame(img_data)
    img_data_df['date'] = pd.to_datetime(img_data_df['date'])
    img_data_df['year'] = img_data_df.date.dt.year
    img_data_df['month'] = img_data_df.date.dt.month
    img_data_df['location'].fillna('Others', inplace=True)
    img_data_df.to_csv('img_data.csv', index=False)
    
def byDate():
    # Organazing by Date
    # Create new dataframe for year and month information
    folder_df =  img_data_df[['year', 'month']].copy()
    folder_df = folder_df.drop_duplicates(subset=['year', 'month'], keep='last').sort_values(by=['year', 'month']).reset_index(drop=True)
    folder_df = folder_df.dropna()
    
    # Source Folder path 
    src_dir = folderPath.get()
    
    # Destination Folder path 
    dest_dir = folderPath2.get()
    
    # mode 
    mode = 0o666

    #create directory for years and months if it is not exist
    for index, folder_row in folder_df.iterrows():
    
        # Directory 
        years = str(folder_row[0])
        months = str(folder_row[1])
      
        # Path 
        years_path = dest_dir +'/' + years 
        months_path = years_path +'/' + months 

        # Create the directory with mode 0o666 
        if not os.path.exists(years_path):
            os.mkdir(years_path, mode) 
    
        if not os.path.exists(months_path):
            os.mkdir(months_path, mode) 

    #Copy images from source to target directory
    for index, img_row in img_data_df.iterrows():
    
        # Directory 
        names = str(img_row[0])
        years = str(img_row[8])
        months = str(img_row[9])
    
        source_path = src_dir +'/' + names
        dest_path = dest_dir +'/' + years +'/' + months 
    
        if (btnChoice.get() == 'Copy'):
            shutil.copy(source_path, dest_path)
        elif (btnChoice.get() == 'Move'):
            shutil.move(source_path, dest_path)
    
def byLocation():
    # Organazing by Location
    # Create new dataframe for location information
    loc_df = img_data_df[['location']].copy()
    loc_df = loc_df.drop_duplicates().sort_values(by=['location']).reset_index(drop=True)

    # Source Folder path 
    src_dir = folderPath.get()
    
    # Destination Folder path 
    dest_dir = folderPath2.get()
   
    # mode 
    mode = 0o666
    
    # Path 
    locPath = dest_dir +'/ByLocation'

    # Create the directory with mode 0o666 
    if not os.path.exists(locPath):
        os.mkdir(locPath, mode) 

    #create directory for locations if it is not exist
    for index, loc_row in loc_df.iterrows():
    
        # Directory 
        imgLocs = str(loc_row[0])  

        # Path 
        filePath = locPath +'/' + imgLocs 

        # Create the directory 
        # with mode 0o666 
        if not os.path.exists(filePath):
            os.mkdir(filePath, mode) 

    #Copy images from source to destionation folders
    for index, loc_img_row in img_data_df.iterrows():
    
        # Directory 
        names = str(loc_img_row[0])
        locs = str(loc_img_row[7])
    
        source_path = src_dir +'/' + names
        dest_path = locPath +'/' + locs
    
        if (btnChoice.get() == 'Copy'):
            shutil.copy(source_path, dest_path)
        elif (btnChoice.get() == 'Move'):
            shutil.move(source_path, dest_path)
            
def getFolderPath():
    folder_selected = filedialog.askdirectory()
    folderPath.set(folder_selected)
    
def getFolderPath2():
    folder_selected = filedialog.askdirectory()
    folderPath2.set(folder_selected)

def doStuff():
    folder = folderPath.get()
    print("Doing stuff with folder", len(folder))

def doButtonClick():
    folder1 = folderPath.get()
    folder2 = folderPath2.get()    
    if (len(folder1) == 0 or len(folder2) == 0):
        messagebox.showwarning("Warning","Please fill out Source and Destination Folders ")

    # Calling function to create data frame
    create_df()

    # Calling function to process button choice
    rb_v = r1_v.get()
    if (rb_v == 0):
        byDate()
        msg = btnChoice.get() + ' by Date processing is completed'
        messagebox.showinfo("Information",msg)
    elif (rb_v == 1):
        byLocation()
        msg = btnChoice.get() + ' by Location processing is completed'
        messagebox.showinfo("Information",msg)

def doCopy():
    btnChoice.set('Copy')
    doButtonClick()
    
def doMove():
    btnChoice.set('Move')
    doButtonClick()
    
def doLocation():
    # Create new dataframe for geo information
    img_data_df = pd.read_csv('img_data.csv')
    
    geo_df = img_data_df[['lat','lon','geo','location']].copy()
    geo_df = geo_df.drop_duplicates(subset=['location'], keep='last').sort_values(by=['location']).reset_index(drop=True)
    geo_df = geo_df.dropna()

    #Define coordinates of where we want to center our map
    lat_mean = geo_df['lat'].mean()
    lon_mean = geo_df['lon'].mean()
    boulder_coords = [lat_mean, lon_mean]

    #Create the map
    my_map = folium.Map(location = boulder_coords, zoom_start = 2)

    #geo_df.to_csv('geo_df.csv', index=False)

    for index, geo_row in geo_df.iterrows():
        # Directory 
        lats = geo_row[0]
        lons = geo_row[1]
        locs = str(geo_row[3])
        if (locs !='nan') :
            #Add markers to the map
            folium.Marker(location=[lats,lons], popup = locs).add_to(my_map)

    #Display the map

    app = QtWidgets.QApplication(sys.argv)
    data = io.BytesIO()
    my_map.save(data, close_file=False)

    w = QtWebEngineWidgets.QWebEngineView()
    w.setHtml(data.getvalue().decode())
    w.resize(640, 480)
    w.show()

    sys.exit(app.exec_())

    
folderPath = StringVar(0)
a = Label(gui, text="Source Folder", font='Helvetica 10 bold')
a.grid(row=1,column = 0, sticky=W, padx=10, pady=0)
E = Entry(gui,textvariable=folderPath, width=75)
E.grid(row=2,column=0, sticky=W, padx=10, pady=0)
btnFind = ttk.Button(gui, text="Browse Folder",command=getFolderPath)
btnFind.grid(row=2,column=1, sticky=W)

folderPath2 = StringVar()
b = Label(gui, text="Destination Folder", font='Helvetica 10 bold')
b.grid(row=3,column = 0, sticky=W, padx=10, pady=0)
F = Entry(gui,textvariable=folderPath2, width=75)
F.grid(row=4,column=0, sticky=W, padx=10, pady=0)
btnFind2 = ttk.Button(gui, text="Browse Folder",command=getFolderPath2)
btnFind2.grid(row=4,column=1, sticky=W)

r1_v = IntVar()
rbt = Label(gui, text="Organize photos by", font='Helvetica 10 bold')        
rbt.grid(row=5,column = 0, sticky=W, padx=10, pady=5)

r1 = ttk.Radiobutton(gui, text='Date', variable=r1_v, value=0)
r1.grid(row=6,column=0, sticky=W, padx=10, pady=0) 

r2 = ttk.Radiobutton(gui, text='Location', variable=r1_v, value=1)
r2.grid(row=7,column=0, sticky=W, padx=10, pady=0) 

loc = ttk.Button(gui, text="Locations on the Map", command=doLocation)
loc.grid(row=7,column=0, sticky=W, padx=180, pady=0)

btnChoice = StringVar(0)
c = ttk.Button(gui ,text='COPY to Destination', command=doCopy)
c.grid(row=8,column=0, sticky=W, padx=30, pady=50)

d = ttk.Button(gui, text="MOVE to Destination", command=doMove)
d.grid(row=8,column=0, sticky=W, padx=180, pady=50)

gui.mainloop()